### 1. 라이브러리 로드

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance 

### 2. 대표폰트 계산

#### 2.1 폰트 이미지 벡터, 클러스터링 결과가 있는 파일 로드

In [3]:
df = pd.read_csv('../Font Clustering/font_clustering_result.csv')
df

,fontname,0,1,2,3,4,5,6,7,8,...,8183,8184,8185,8186,8187,8188,8189,8190,8191,cluster
0,10X10,0.249864,0.032769,0.153393,0.167023,0.048019,0.036557,0.151577,0.813157,0.251208,...,0.487822,0.006675,1.962767,0.641799,1.256014,1.274865,0.372895,0.390966,0.076515,0
1,10X10Bold,0.306329,0.041129,0.119644,0.192284,0.026035,0.019363,0.180128,0.721529,0.400074,...,0.581136,0.006415,1.920637,0.175987,1.227234,1.901613,0.384293,0.395105,0.082669,0
2,12롯데마트드림Bold,0.234252,0.008728,0.000000,0.287013,0.030751,0.035574,0.089000,0.949538,0.449302,...,0.715202,0.034027,1.550845,0.874185,0.749693,1.868115,0.091056,0.347581,0.353270,0
3,12롯데마트드림Light,0.463416,0.078549,0.028541,0.090770,0.075423,0.026523,0.041908,1.454447,0.293393,...,0.513337,0.109182,1.826288,0.831087,1.140763,1.200037,0.168479,0.165177,0.552506,1
4,12롯데마트드림Medium,0.352521,0.042633,0.019750,0.187089,0.087354,0.089137,0.012848,1.108636,0.365287,...,0.654345,0.019371,1.673759,0.831088,1.085076,1.480671,0.113763,0.215111,0.324763,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,해수체L,0.713258,0.110830,0.036123,0.069608,0.000000,0.354927,0.017896,1.453221,0.494302,...,0.704711,0.014192,2.137889,0.852427,1.203440,0.851186,0.431453,0.160215,0.101563,1
179,해수체M,0.606477,0.053838,0.051595,0.146851,0.006021,0.365343,0.048289,1.245699,0.341356,...,0.708893,0.022360,1.896775,0.796216,1.035430,0.954128,0.337135,0.134271,0.117077,1
180,헬스셋고딕Bold,0.382735,0.212755,0.158224,0.581540,0.004145,0.296222,0.196923,1.672214,1.107548,...,0.775121,0.033765,1.367147,0.659745,0.355626,2.255158,0.356245,0.466157,0.937279,1
181,헬스셋고딕Light,0.580152,0.453038,0.085047,0.490497,0.010695,0.477680,0.094734,1.674429,0.840052,...,0.665388,0.086993,1.697156,0.883348,0.796820,1.640746,0.581760,0.574731,1.003001,1


#### 2.2 대표폰트 계산

##### 2.2.1 코사인 유사도(중심점과 가장 가까운 폰트)

In [252]:
for cluster_num in range(4):
  cluster_fonts_name = df[df['cluster']==cluster_num].iloc[:, 0]
  cluster_df = df[df['cluster']==cluster_num].iloc[:, 1:-1]
  cluster_centroid = cluster_df.median().to_numpy().reshape(-1, 1).reshape((1, -1))

  centroid_df_distance = []

  for idx, row in cluster_df.iterrows():
    dist = cosine_similarity(cluster_centroid, cluster_df.loc[[idx]].to_numpy())  
    centroid_df_distance.append(dist[0][0])  
    
  cluster_df['distance'] = centroid_df_distance
  cluster_df.insert(0, 'fontname', cluster_fonts_name)
  repres_font = cluster_df.iloc[cluster_df['distance'].argmax()]['fontname']
  
  print('cluster', cluster_num, ' 대표폰트 : ', repres_font)

cluster 0  대표폰트 :  GOYANGDEOGYANG B
cluster 1  대표폰트 :  KoPubWorld Dotum Medium
cluster 2  대표폰트 :  KBIZ한마음명조 M
cluster 3  대표폰트 :  netmarbleM


##### 2.2.2 유클리드 거리(중앙값과 가장 가까운 폰트)

In [253]:
for cluster_num in range(4):
  cluster_fonts_name = df[df['cluster']==cluster_num].iloc[:, 0]
  cluster_df = df[df['cluster']==cluster_num].iloc[:, 1:-1]
  cluster_median = cluster_df.median().values.reshape(-1, 1)

  median_df_distance = []
  
  for idx in range(len(cluster_df)):
    dist = distance.euclidean(cluster_median, cluster_df.iloc[idx].values.reshape(-1, 1))  
    median_df_distance.append(dist)

  cluster_df['distance'] = median_df_distance
  cluster_df.insert(0, 'fontname', cluster_fonts_name)
  repres_font = cluster_df.iloc[cluster_df['distance'].argmin()]['fontname']

  print('cluster', cluster_num, ' 대표폰트 : ', repres_font)

cluster 0  대표폰트 :  GOYANGDEOGYANG B
cluster 1  대표폰트 :  KoPubWorld Dotum Medium
cluster 2  대표폰트 :  KoPubWorld Batang Light
cluster 3  대표폰트 :  netmarbleM
